Here we outline the process of using the YOLOv5 code in google collab. Very simple!!  

# **Introduction and Setup**
### Here we outline the process for using the YOLOv5 Network

First mount the Gdrive to the Google Collab session in order to save results. Next the necessary code is imported from the Ultralytics YOLOv5 repository.

The repository can be found here: https://github.com/ultralytics/yolov5

In [ ]:
# Mount google drive for saving the results

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Clone in YOLOv5 repo from ultralytics
!git clone https://github.com/ultralytics/yolov5.git
%cd /content

!pip install -r yolov5/requirements.txt  # install dependencies
%cd yolov5


from datetime import date
today = date.today().strftime('%Y%m%d')
import torch
print('Setup complete on %s. Using torch %s %s' % (today, torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# **Upload Custom Data**

Next we create directories to upload the training validation and test sets also their labels.

In [ ]:
# make train, validation, and testing directories to upload images and labels

%cd /content
%mkdir /content/train
%mkdir /content/train/images  
%mkdir /content/train/labels

%mkdir /content/valid
%mkdir /content/valid/images
%mkdir /content/valid/labels

%mkdir /content/test
%mkdir /content/test/images

print('Directories created.')

# **.yaml File Configuration**

The yaml file is a simple configuration file. It allows the YOLOv5 code to know the path to your training and validation data.  The number of detection classes and what the class names are. 

In our case all of the detections are on single classes i.e. triangle or rectangle. 

YAML files are very easy to write in a simple text editor, or using python. Below we provide an example using python:

In [ ]:
# Writing an example YAML file for Triangles.
%cd /content
with open('data.yaml', 'w') as outfile:
  
  outfile.write('train: ../train/images\n') # path from /yolov5/train.py to directory with uploaded training data 
  outfile.write('val: ../valid/images\n') # path from /yolov5/train.py to directory with validation data
  
  outfile.write('\n')

  outfile.write('nc: 1\n')
  outfile.write("names: ['Rec']\n") ### edit accordingly to the structure or object

print('Yaml file written.')

In [ ]:
# check yaml datafile 
# if uploading one either rename or adjust namespace accordingly
%cd /content
%cat data.yaml

# **Review Input Data**

Here we review all the data which has been uploaded to ensure we have the correct information for training.  This will also help to estimate batch sizing, training time, etc...

In [ ]:
%cd /content
# view the breakdown of the images that you have for training and testing
# define number of classes based on YAML
import yaml
import os

os.chdir('/content')
with open("data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
print("There is n = %s classes in this data set" %num_classes)
print('---------------------------------------------------------------')

# prints the number of trining images
train_imgs = sorted(os.listdir('/content/train/images'))
print('There are %i images ready for training' % len(train_imgs))
for img in sorted(train_imgs):
  print(img)
print('---------------------------------------------------------------')

# prints the number of validation images
train_imgs = sorted(os.listdir('/content/valid/images'))
print('There are %i images ready for validation' % len(train_imgs))
for img in sorted(train_imgs):
  print(img)
print('---------------------------------------------------------------')

# prints the number of test images ready for detection
test_imgs = sorted(os.listdir('/content/test/images'))
print('There are %i images ready for detection' %len(test_imgs))
for img in sorted(test_imgs):
  print(img)
print('---------------------------------------------------------------')

# **Define the model configuration**

Here is where you can define or write a custom model or use the YOLOv5 default models.

There are 4 different default models which can be found in the /yolov5/models  directory:

1. Yolov5s
2. Yolov5m
3. Yolov5l
4. Yolov5x

Each model's performance based on benchmark datasets, can be reviewed in the ultralytics yolov5 repository. Generally speaking they trade speed for accuracy with Yolov5s being the fastest and Yolov5x being the slowest.

Here we utilize the Yolov5s for our experiments.  

In [ ]:
%cd /content

%cat yolov5/models/yolov5s.yaml # prints the model output for reference

# **Train the Network**

Here is where we imput our training and validation data into the Yolov5s framework. 

Default parameters can be viewed in the /yolov5/train.py file. 

Some changes we made were:

* Adjusting batch size to 10 in order to fit cache into GPU memory.
* Training on 1280 images. (default is 960 and image size should remain a multiple of 32).
* Ran for 500 epochs 
* Run with multi-scale

The weights are stored according to the `--name` argument as a pytorch file (.pt)

In [ ]:
#### TRAINING STEP, TIME CONSUMING ####
%%time 
%cd /content/yolov5/

import time
start = time.time() # Record training time

#Parameters can be changed here or in command below
model = 'full_rectangle'
batch_size = 10
image_size = 1280
epochs = 1000

#The name by which the data will be saved into yolov5 directory
name = f'{today}_{model}_{image_size}_{epochs}'

# weights and trainign data are stored in the appropriate subdirectory in the /content/yolov5/runs directory
!python train.py --name {name} --batch 10 --img-size 1280 --epochs 1000 --data '../data.yaml' --cfg ./models/yolov5s.yaml --weights ''  --cache | tee {name}_train_log.txt

fin = time.time()- start
hrs = fin*(1/60)**2

# Store the training time in a separate file.
with open(f'{name}_train_time.txt', 'w') as outfile:
  print(f'Training completed in {hrs:0.3f} hours',file=outfile)
  print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'),file=outfile)


# **Test Set Detection**

Here we run the detection algorithm. Default parameters can be viewed in the /yolov5/train.py file. 

Some changes we made were:
* IoU threshold increased to 0.5
* Confidence threshold increased to 0.8
* Image size the same as training i.e. 1280  

In [ ]:
#### Detection Step ####


# detection data is stored in the appropriate subdirectory in the /content/yolov5/runs directory
%cd /content/yolov5

#Parameters can be changed here or in command below
model = 'full_triangle'
image_size = 1280
iou = 0.5
confidence = 0.8

#The name by which the data will be saved into yolov5 directory
name = f'{today}_{model}_{image_size}'

#Execute the detection script
!python detect.py --name {name} --weights /content/yolov5/weights/best.pt --save-txt --img {image_size} --iou {iou} --conf {confidence} --source ../test/images | tee {name}_detect_results.txt

# **Save Results to Google Drive** 
*Only available if the google drive mount key was used in the first code segment* 

This code segement saves the copies the current yolov5 directory into google drive. 

!!Caution data will be overwritten!!

In [ ]:
# predictions are found under the output folder and weights are stored in the pytorch file (".pt")
%cp  -r /content/yolov5 /content/gdrive/My\ Drive 

print("Copied to google drive")

# **Clear Completed Data**

This code segment programatically clears the run, training, validation, and testing data that currently resides in the collab session.  

It will leave the Gdrive untouched!

*Caution before clearing run data as it holds weights and detection results*

In [ ]:
# uncomment these to clear the directories and upload new images and lables

import os 
import shutil

os.chdir('/content/yolov5/runs')  
for dir in os.listdir(): #### make sure data is saved before it is erased!!
  shutil.rmtree(dir)     #### this code block removes previous training and detection runs 

os.chdir('/content/train/images/')
for file in os.listdir():
  os.remove(file)

os.chdir('/content/train/labels')
for file in os.listdir():
  os.remove(file)

os.chdir('/content/valid/labels')
for file in os.listdir():
  os.remove(file)

os.chdir('/content/valid/images')
for file in os.listdir():
  os.remove(file)

os.chdir('/content/test/images')
for file in sorted(os.listdir()):
  os.remove(file)

print('Done.')